In [54]:
# 1.Daily covid cases

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime
city_popu=pd.read_hdf('./Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

In [56]:
df=pd.read_excel(r'DataSource/airports.xlsx')
df=df.drop(columns=['Unit'])
df.to_hdf('./Processed_Data/airports.h5',key='raw')
df.tail(2)

,城市,机场,2019-01-01 00:00:00,2019-01-02 00:00:00,2019-01-03 00:00:00,2019-01-04 00:00:00,2019-01-05 00:00:00,2019-01-06 00:00:00,2019-01-07 00:00:00,2019-01-08 00:00:00,...,2022-12-17 00:00:00,2022-12-18 00:00:00,2022-12-19 00:00:00,2022-12-20 00:00:00,2022-12-21 00:00:00,2022-12-22 00:00:00,2022-12-23 00:00:00,2022-12-24 00:00:00,2022-12-25 00:00:00,2022-12-26 00:00:00
231,扎兰屯,成吉思汗,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,4.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,2.0,2.0
232,中卫,沙坡头,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0


In [57]:
df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
df_covid.tail(2)

,city,progress,peak,end,城市,常住人口,progress_2
295,聊城,回落至34%,2022/12/20,2023/03/07,聊城,201,34
296,宿迁,回落至41%,2022/12/21,2023/03/08,宿迁,310,41


In [58]:
# TY 机场 vs. 2019
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
riqi='2022-12-26'


ap=pd.read_hdf('./Processed_Data/airports.h5',key='raw')
ap=ap.groupby(by='城市').sum().reset_index()
lastcol=ap.columns[-1]
ap=ap.sort_values(by=lastcol,ascending=False).head(50)
ap=pd.melt(ap,id_vars=['城市'],var_name='date',value_name='air_traffic')
ap=ap[ap['air_traffic']>0]

ap['date'] = pd.to_datetime(ap['date']).dt.date
ap=ap.merge(df_covid,on=['城市'],how='left')
ap=ap.sort_values(by=['城市','date'],ascending=True)
ap['date']=ap['date'].astype('str')

def getdt(d):
    return str(d)[-5:]
ap['dt']=ap['date'].apply(getdt)

ap['ap_last7d_mean']=ap.groupby('城市')['air_traffic'].transform(lambda x: x.rolling(7, 3).mean())
ap['ap_WoW']=ap['ap_last7d_mean']/ap.groupby('城市')['ap_last7d_mean'].transform(lambda x: x.shift(periods=7)) -1

c1=ap['date'].str.contains('2019')
ap_19=ap[c1]
ap_19=ap_19[['城市', 'dt','ap_last7d_mean']]
ap_19=ap_19.rename(columns={'ap_last7d_mean':'ap_19_rolling_mean'})

c2=ap['date'].str.contains('2022')
ap_22=ap[c2]
ap_22=ap_22.merge(ap_19,how='left',on=['城市','dt'])

ap_22['ap_status']=ap_22['ap_last7d_mean']/ap_22['ap_19_rolling_mean']

ap_22['covid_rank']=ap_22.groupby(by=['date'])['progress_2'].rank(method="dense", ascending=False)
o=ap_22[ap_22['ap_status']<2][ap_22['date']==riqi][['城市','covid_rank','ap_status']] 

o['size']=np.nan
o['group']='C'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='A'

c2=o['城市'].isin(jidui_city)
o.loc[c2,'group']='B'

o=o.sort_values(by='group')

o.to_clipboard(index=False,header=None)

print(riqi)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-12-26


In [59]:
# TY 机场数据 WoW
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

ap_22['covid_rank']=ap_22.groupby(by=['date'])['progress_2'].rank(method="dense", ascending=False)
o=ap_22[ap_22['date']==riqi][['城市','covid_rank','ap_WoW']] 

o['size']=np.nan
o['group']='C'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='A'

c2=o['城市'].isin(jidui_city)
o.loc[c2,'group']='B'

o=o.sort_values(by='group')

o.to_clipboard(index=False,header=None)
print(riqi)

2022-12-26


In [60]:
citylist=['北京','石家庄','成都','重庆','武汉','郑州']
c3=ap_22['城市'].isin(citylist)
c4=ap_22['date']>'2022-03-01'
show=ap_22[c3&c4]


show=pd.pivot_table(show,index='date',columns='城市',values='ap_status',aggfunc='mean').reset_index()
show['sum']=np.nan

show=show[['date','sum']+citylist]
show.to_clipboard(index=False)